<a href="https://colab.research.google.com/github/ManelBoucenna/AI-s-code-repository/blob/master/SQL_beginner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.cloud import bigquery

In [3]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
# Create a "Client" object
client = bigquery.Client( project="bigquery-public-data")

In [0]:
dataset_ref = client.dataset("hacker_news")

In [0]:
dataset = client.get_dataset(dataset_ref)

Every dataset is just a collection of tables. You can think of a dataset as a spreadsheet file containing multiple tables, all composed of rows and columns.

In [10]:
# List all the tables in the "hacker_news" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there are four!)
for table in tables:  
    print(table.table_id)

comments
full
full_201510
stories


In [0]:
# Construct a reference to the "full" table
table_ref = dataset_ref.table("full")

# API request - fetch the table
table = client.get_table(table_ref)

In [12]:
# Print information on all the columns in the "full" table in the "hacker_news" dataset
table.schema


[SchemaField('title', 'STRING', 'NULLABLE', 'Story title', ()),
 SchemaField('url', 'STRING', 'NULLABLE', 'Story url', ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Story or comment text', ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', ()),
 SchemaField('by', 'STRING', 'NULLABLE', "The username of the item's author.", ()),
 SchemaField('score', 'INTEGER', 'NULLABLE', 'Story score', ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', ()),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'Timestamp for the unix time', ()),
 SchemaField('type', 'STRING', 'NULLABLE', 'Type of details (comment, comment_ranking, poll, story, job, pollopt)', ()),
 SchemaField('id', 'INTEGER', 'NULLABLE', "The item's unique id.", ()),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', ()),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', 'Number of story or poll descendants', ()),
 SchemaField('ranking', 'INTEGER', 'NULLABLE', 'Comment ranking', ()),
 SchemaField(

In [14]:
# Preview the first five lines of the "full" table
client.list_rows(table, max_results=3).to_dataframe()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,I stick with Hover which is excellent and Slic...,None,bluedanieru,None,1303818015,2011-04-26 11:40:15+00:00,comment,2484747,2484728,None,None,None
1,None,None,"Seems like nice idea, good luck!",None,pie6k,None,1566572328,2019-08-23 14:58:48+00:00,comment,20778300,20777674,None,None,None
2,None,None,None,None,None,None,1467030222,2016-06-27 12:23:42+00:00,comment,11985686,11981400,None,None,True


In [15]:
# Preview the first five entries in the "by" column of the "full" table
client.list_rows(table, selected_fields=table.schema[:1], max_results=5).to_dataframe()


,title
0,None
1,None
2,None
3,None
4,None


# SQL Queries

Note that when writing an SQL query, the argument we pass to FROM is not in single or double quotation marks (' or "). It is in backticks (`).

In [54]:
client = bigquery.Client(project="bigquery-public-data")

# Construct a reference to the "openaq" dataset
dataset_ref = client.dataset("openaq")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# List all the tables in the "openaq" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there's only one!)
for table in tables:  
    print(table.table_id)

global_air_quality


In [0]:
query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

In [56]:
# Set up the query
query_job = client.query(query)

Forbidden: ignored

In [0]:
us-cities = query_job.to_dataframe()

In [0]:
us-cities.value_counts().head()


# Working with big datasets
How to estimate the size of a query?

In [0]:
query=
"""
        SELECT score, title
        FROM `bigquery-public-data.hacker_news.full`
        WHERE type = "job" 
"""
# Create a QueryJobConfig object to estimate size of query without running it
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query, job_config=dry_run_config)

print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

You can also specify a parameter when running the query to limit how much data you are willing to scan. Here's an example with a low limit.

In [0]:
# Only run the query if it's less than 1 MB
ONE_MB = 1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_MB)

# Set up the query (will only run if it's less than 1 MB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
safe_query_job.to_dataframe()